基于Kaggle网站的数据集，对新冠肺炎进行EDA和可视化。

In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# 绘制地图
import folium 
from folium import plugins

# Graphics in retina format 
%config InlineBackend.figure_format = 'retina' 

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = 8, 5

# Disable warnings 
import warnings
warnings.filterwarnings('ignore')

# Import Data

直接从数据源下载，一直报错。尝试netsh winsock reset并重启，仍然无效。将数据下载下来再进行分析。

Kaggle中的数据不全，数据集的时间截止到：2020-02-17

而到文档更新时间2020-3-3,韩国已经确诊超4千，这里只有几十个。

In [53]:
# urlopen error [WinError 10060]
# confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')


In [54]:
# confirmed = pd.read_csv('novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')
# confirmed.head()

In [13]:
# recovered = pd.read_csv('novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv')
# recovered.head()

In [14]:
# deaths = pd.read_csv('novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv')
# deaths.head()

In [15]:
# confirmed.shape, recovered.shape, deaths.shape

In [16]:
# confirmed.isna().sum()[confirmed.isna().sum() > 0]

In [17]:
# recovered.isna().sum()[recovered.isna().sum() > 0]

In [18]:
# deaths.isna().sum()[deaths.isna().sum() > 0]

In [19]:
# 将表中有缺失值的数据使用Unknown填充。
# confirmed = confirmed.fillna('unknow')
# recovered = recovered.fillna('unknow')
# deaths = deaths.fillna('unknow')

In [192]:
data = pd.read_csv("novel-corona-virus-2019-dataset/2019_nCoV_data.csv")
# data.head()

**缺失值填充。**

In [193]:
data.isna().sum()[data.isna().sum() > 0]

Province/State    462
dtype: int64

看原始数据发现，在Province/State这列，有462个空白缺失值。使用Unknown填充。


In [194]:
data = data.fillna('Unknow')

这里的数据有**重复**的，需要去除那些不是最新时间的数据。



In [195]:
data['Last Update'] = data['Last Update'].apply(pd.to_datetime)
data['Date'] = data['Date'].apply(pd.to_datetime)
data.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22 12:00:00,Anhui,China,2020-01-22 12:00:00,1.0,0.0,0.0
1,2,2020-01-22 12:00:00,Beijing,China,2020-01-22 12:00:00,14.0,0.0,0.0
2,3,2020-01-22 12:00:00,Chongqing,China,2020-01-22 12:00:00,6.0,0.0,0.0
3,4,2020-01-22 12:00:00,Fujian,China,2020-01-22 12:00:00,1.0,0.0,0.0
4,5,2020-01-22 12:00:00,Gansu,China,2020-01-22 12:00:00,0.0,0.0,0.0


In [196]:
d = data['Date'][-1:].astype('str')
year = int(d.values[0].split('-')[0])
month = int(d.values[0].split('-')[1])
day = int(d.values[0].split('-')[2].split()[0])

china = data[data['Country']=='Mainland China']

from datetime import date
data_latest = data[data['Date'] > pd.Timestamp(date(year,month,day))]

china_latest = data_latest[data_latest['Country']=='Mainland China']

data_latest.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
1644,1645,2020-02-17 22:00:00,Hubei,Mainland China,2020-02-17 23:13:06,59989.0,1789.0,7862.0
1645,1646,2020-02-17 22:00:00,Guangdong,Mainland China,2020-02-17 12:43:02,1322.0,4.0,524.0
1646,1647,2020-02-17 22:00:00,Henan,Mainland China,2020-02-17 14:53:02,1246.0,16.0,509.0
1647,1648,2020-02-17 22:00:00,Zhejiang,Mainland China,2020-02-17 11:23:06,1171.0,0.0,507.0
1648,1649,2020-02-17 22:00:00,Hunan,Mainland China,2020-02-17 11:43:01,1006.0,3.0,498.0


In [197]:
data_latest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 1644 to 1718
Data columns (total 8 columns):
Sno               75 non-null int64
Date              75 non-null datetime64[ns]
Province/State    75 non-null object
Country           75 non-null object
Last Update       75 non-null datetime64[ns]
Confirmed         75 non-null float64
Deaths            75 non-null float64
Recovered         75 non-null float64
dtypes: datetime64[ns](2), float64(3), int64(1), object(2)
memory usage: 5.3+ KB


In [198]:
data = data_latest

# EDA & Viz

## 全球概览

截止到数据集中的时间，全球共有30个国家有确诊新冠肺炎病例。

In [178]:
countries = data['Country'].unique().tolist()
print("Names of Country: \n", countries)
print("\n")
print("Number of countries:", len(countries))

Names of Country: 
 ['Mainland China', 'Others', 'Singapore', 'Japan', 'Hong Kong', 'Thailand', 'South Korea', 'Malaysia', 'Taiwan', 'Germany', 'Vietnam', 'France', 'Macau', 'UK', 'United Arab Emirates', 'Australia', 'Canada', 'India', 'Italy', 'Philippines', 'Russia', 'Spain', 'US', 'Belgium', 'Cambodia', 'Egypt', 'Finland', 'Nepal', 'Sri Lanka', 'Sweden']


Number of countries: 30


我们发现国家名称中有China和Mainland China，替换掉China。

还有Others这一列，存储的是Cruise Ship，替换。

In [179]:
data['Country'].replace({'China': 'Mainland China'},inplace=True)
data['Country'].replace({'Others': 'Diamond Princess Cruise Ship'},inplace=True)
countries = data['Country'].unique().tolist()
print(countries)

['Mainland China', 'Diamond Princess Cruise Ship', 'Singapore', 'Japan', 'Hong Kong', 'Thailand', 'South Korea', 'Malaysia', 'Taiwan', 'Germany', 'Vietnam', 'France', 'Macau', 'UK', 'United Arab Emirates', 'Australia', 'Canada', 'India', 'Italy', 'Philippines', 'Russia', 'Spain', 'US', 'Belgium', 'Cambodia', 'Egypt', 'Finland', 'Nepal', 'Sri Lanka', 'Sweden']




查看总数据量。

In [183]:
cases = pd.DataFrame(data.groupby('Country')['Confirmed', 'Deaths', 'Recovered'].sum())

print('Till 2020-2-17, Gloabal Confirmed Cases are : ', cases['Confirmed'].sum())

print('Till 2020-2-17, Gloabal Deaths Cases are : ', cases['Deaths'].sum())

print('Till 2020-2-17, Gloabal Recovered Cases are : ', cases['Recovered'].sum())

cases.sort_values(by = 'Confirmed', ascending = False)

Till 2020-2-17, Gloabal Confirmed Cases are :  73260.0
Till 2020-2-17, Gloabal Deaths Cases are :  1868.0
Till 2020-2-17, Gloabal Recovered Cases are :  12583.0


,Confirmed,Deaths,Recovered
Country,,,
Mainland China,72364.0,1863.0,12455.0
Diamond Princess Cruise Ship,454.0,0.0,0.0
Singapore,77.0,0.0,24.0
Japan,66.0,1.0,12.0
Hong Kong,60.0,1.0,2.0
Thailand,35.0,0.0,15.0
South Korea,30.0,0.0,10.0
Malaysia,22.0,0.0,7.0
Taiwan,22.0,1.0,2.0


出现死亡的有6个国家。

In [184]:
temp = cases[cases['Deaths'] > 0]
temp.style.background_gradient(cmap='Pastel1_r')

,Confirmed,Deaths,Recovered
Country,,,
France,12,1,4
Hong Kong,60,1,2
Japan,66,1,12
Mainland China,72364,1863,12455
Philippines,3,1,1
Taiwan,22,1,2


## 绘制世界确诊地图

这里参考了Kaggle中的方式，以及绘制地图博客：https://www.cnblogs.com/traditional/p/12386907.html

In [185]:
world_coordinates = pd.read_csv('novel-corona-virus-2019-dataset/world_coordinates.csv')

# 相应的要将坐标数据中的国家替换

world_coordinates['Country'].replace({'China': 'Mainland China'},inplace=True)

world_data = pd.merge(world_coordinates, cases, on='Country')
world_data.head()

,Code,Country,latitude,longitude,Confirmed,Deaths,Recovered
0,AE,United Arab Emirates,23.424076,53.847818,9.0,0.0,4.0
1,AU,Australia,-25.274398,133.775136,15.0,0.0,10.0
2,BE,Belgium,50.503887,4.469936,1.0,0.0,1.0
3,CA,Canada,56.130366,-106.346771,8.0,0.0,1.0
4,CN,Mainland China,35.861660,104.195397,72364.0,1863.0,12455.0


In [109]:
# world_data.info()
# 想根据多少来显示圆圈大小，方法不可行，差距太大
# tmp = world_data['Confirmed'].max()  - world_data['Confirmed'].min()
# world_data['ConfirmedNew'] = (world_data['Confirmed'] - world_data['Confirmed'].min()) / tmp

# world_data['ConfirmedNew'].max(), world_data['ConfirmedNew'].min()

In [140]:
# for lat, lon, value, name in zip(world_data['latitude'], world_data['longitude'], world_data['Confirmed'], world_data['Country']):
#    print(lat, lon)

# world_map = folium.Map(location=[10, -20], zoom_start=2.3,tiles='Stamen Toner')
world_map = folium.Map(location=[10, -20], zoom_start=2.3)
# len(world_data)

for i in range(len(world_data)):
    
    folium.CircleMarker(location = [world_data['latitude'][i], world_data['longitude'][i]], 
                                    # popup = popup
                       radius = 10,
                
                        color = 'red',
                        
                        fill_color='red',
                        fill_opacity=0.7 ).add_to(world_map)
                        
# world_map

In [186]:
world_map = folium.Map(location=[10, -20], zoom_start=2.3,tiles='Stamen Toner')
 

for lat, lon, value, name in zip(world_data['latitude'], world_data['longitude'], world_data['Confirmed'], world_data['Country']):
    folium.CircleMarker(location = [lat, lon],
                        radius = 10,
                        # radius = value_ra,
                        # 使用变量显示时，地图显示错误
                        # 使用上面的HTML格式显示
                        # popup = ('<strong>Country</strong>: ' + str(name).capitalize() + '<br>'
                        #          '<strong>Confirmed Cases</strong>: ' + str(value) + '<br>'),
                        # popup = str(name),
                        # popup='<p style="color: green"></p> '+ str(name),
                        # popup = '<b>Country:{}</b></br> <b>Cases:{}</b></br>'.format(name, value),
                        color = 'red',
                        
                        fill_color='red',
                        fill_opacity=0.7 
                       ).add_to(world_map)
world_map.save('confirmed_world_map.html')
world_map

In [199]:

china_date = china.groupby(['Date', 'Province/State'])['Confirmed', 'Deaths', 'Recovered'
                                                      ].max()
china_date = china_date.reset_index()
# china_map['size'] = china_map['Confirmed'].pow(0.5)
china_date['Date'] = pd.to_datetime(china_date['Date'])
china_date['Date'] = china_date['Date'].dt.strftime('%m/%d/%Y')
china_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 5 columns):
Date              801 non-null object
Province/State    801 non-null object
Confirmed         801 non-null float64
Deaths            801 non-null float64
Recovered         801 non-null float64
dtypes: float64(3), object(2)
memory usage: 31.4+ KB


0      01/23/2020
1      01/23/2020
2      01/23/2020
3      01/23/2020
4      01/23/2020
5      01/23/2020
6      01/23/2020
7      01/23/2020
8      01/23/2020
9      01/23/2020
10     01/23/2020
11     01/23/2020
12     01/23/2020
13     01/23/2020
14     01/23/2020
15     01/23/2020
16     01/23/2020
17     01/23/2020
18     01/23/2020
19     01/23/2020
20     01/23/2020
21     01/23/2020
22     01/23/2020
23     01/23/2020
24     01/23/2020
25     01/23/2020
26     01/23/2020
27     01/23/2020
28     01/23/2020
29     01/23/2020
          ...    
771    02/17/2020
772    02/17/2020
773    02/17/2020
774    02/17/2020
775    02/17/2020
776    02/17/2020
777    02/17/2020
778    02/17/2020
779    02/17/2020
780    02/17/2020
781    02/17/2020
782    02/17/2020
783    02/17/2020
784    02/17/2020
785    02/17/2020
786    02/17/2020
787    02/17/2020
788    02/17/2020
789    02/17/2020
790    02/17/2020
791    02/17/2020
792    02/17/2020
793    02/17/2020
794    02/17/2020
795    02/